<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-просмотр-данных" data-toc-modified-id="Загрузка-и-просмотр-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и просмотр данных</a></span></li><li><span><a href="#Обработка-данных" data-toc-modified-id="Обработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обработка данных</a></span><ul class="toc-item"><li><span><a href="#Функция-лемматизации-текста-и-создание-признака-из-лемм" data-toc-modified-id="Функция-лемматизации-текста-и-создание-признака-из-лемм-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Функция лемматизации текста и создание признака из лемм</a></span></li><li><span><a href="#Разделение-выборки-на-train-и-test" data-toc-modified-id="Разделение-выборки-на-train-и-test-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Разделение выборки на train и test</a></span></li></ul></li><li><span><a href="#Cоздание-Tf-idf-признаков" data-toc-modified-id="Cоздание-Tf-idf-признаков-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Cоздание Tf-idf признаков</a></span></li><li><span><a href="#Обучение-и-проверка-моделей,-выбор-лучшей" data-toc-modified-id="Обучение-и-проверка-моделей,-выбор-лучшей-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Обучение и проверка моделей, выбор лучшей</a></span><ul class="toc-item"><li><span><a href="#Разделение-на-признаки-и-целевой-признак" data-toc-modified-id="Разделение-на-признаки-и-целевой-признак-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Разделение на признаки и целевой признак</a></span></li><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#DecisionTreeClassifier" data-toc-modified-id="DecisionTreeClassifier-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>DecisionTreeClassifier</a></span></li><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Проект по классификации тональности текстов

Есть база данных интернет-магазина с комментариями на английском языке, каждому комментарию присвоена метка 0 или 1, где 0 - комментарий позитивный, 1 - комментарий негативный. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Задача: обучить модель классифицировать комментарии на позитивные и негативные.

Метрика качества модели - *F1*, она должна быть не меньше 0.75.

## Загрузка и просмотр данных

In [1]:
from tqdm import tqdm
import pandas as pd
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
pd.options.mode.chained_assignment = None
from sklearn.metrics import f1_score
from pymystem3 import Mystem
m = Mystem()
import re
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import spacy
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
display(df)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


## Обработка данных

In [4]:
df.isna().sum()

text     0
toxic    0
dtype: int64

In [5]:
df.duplicated().sum()

0

### Функция лемматизации текста и создание признака из лемм

In [6]:
nlp = spacy.load('en_core_web_sm')

In [7]:
def LEMM(text):
    sentence = nlp(text)
    
    T = []

    for word in sentence:
        T.append(word.lemma_)
        
    def clear_text(text):
        return " ".join(re.sub(r'[^a-zA-Z ]', ' ', text).split())
    
    Q = []

    for i in range(len(T)):
        Q.append(clear_text(T[i]))
    
    Q = ' '.join(Q)
    
    del T
    
    return Q

In [8]:


W = []

for i in tqdm(range(len(df['text']))):
    W.append(LEMM(df['text'][i]))



100%|██████████| 159571/159571 [48:27<00:00, 54.88it/s]  


In [11]:
df['lemm'] = W

In [12]:
del W

In [13]:
df = df.drop('text', axis=1)

In [14]:
display(df)

,toxic,lemm
0,0,Explanation why the edit make under my userna...
1,0,D aww he match this background colour I be se...
2,0,hey man I be really not try to edit war it b...
3,0,More I can not make any real suggestion on ...
4,0,you sir be my hero any chance you remember ...
...,...,...
159566,0,and for the second time of asking when ...
159567,0,you should be ashamed of yourself that be a h...
159568,0,spitzer Umm there s no actual article for pr...
159569,0,and it look like it be actually you who put on...


### Разделение выборки на train и test

In [15]:
train, test = train_test_split(df, test_size=0.4, random_state=12345) 

## Cоздание Tf-idf признаков 

In [16]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

vectorizer = TfidfVectorizer(stop_words=stopwords)



tf_train = vectorizer.fit_transform(train['lemm'])


tf_test = vectorizer.transform(test['lemm'])

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**vectorizer обучили на тренировочных данных**

## Обучение и проверка моделей, выбор лучшей

### Разделение на признаки и целевой признак

In [17]:
features = tf_train
target = train['toxic']

In [18]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [19]:
features_test = tf_test
target_test = test['toxic']

**Модель с лучшими гиперпараметрами выбираем с помощью метода кросс-валидации, метрика *f1***

### LogisticRegression

Внутри кросс-валидации происходит разбиение переданной в нее выборки на части (треин и валидация). 

Нужно, чтобы при подборе параметров внутри кросс-валидации векторизатор не обучался на валидации. Для этого обучим его каждый раз только на треине (внутри кросс-валидации).

**Подбор гиперпараметров:**

In [28]:
scor_best = 0
C_best = 0

for C in np.arange(6.0, 6.2, 0.1):
    pipe_lr = Pipeline([('tfidf', TfidfVectorizer(stop_words=stopwords)),
     ('model', LogisticRegression(solver='liblinear', C=C, random_state=12345))])
    scor = (cross_val_score(pipe_lr, train['lemm'], train['toxic'], scoring='f1', cv=20)).mean()
    print(scor, C)
    if scor > scor_best:
        scor_best = scor
        C_best = C

        
print('Лучший скор:', scor_best, 'При С:', C_best)

0.7718073369791507 6.0
0.7720400518074783 6.1
0.772379568622079 6.199999999999999
Лучший скор: 0.772379568622079 При С: 6.199999999999999


Метрикой качества готовой модели служит f1 мера.

**Проверка модели на валидационных данных:**

In [30]:
model = LogisticRegression(solver='liblinear', C=C_best, random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("f1:", f1_score(predicted_valid, target_valid))

f1: 0.7591549295774649


**Проверка на тестовых данных:**

In [31]:
predicted_test = model.predict(features_test)
print("f1:", f1_score(predicted_test, target_test))

f1: 0.7643895989422653


### DecisionTreeClassifier

**Подбор гиперпараметров:**

In [32]:
scor_best = 0
depth_best = 0

for depth in range(4, 6):
    
    pipe_lr = Pipeline([('tfidf', TfidfVectorizer(stop_words=stopwords)),
     ('model', DecisionTreeClassifier(random_state=12345, max_depth=depth))])
    scor = (cross_val_score(pipe_lr, train['lemm'], train['toxic'], scoring='f1', cv=20)).mean()
    print(scor, depth)
    if scor > scor_best:
        scor_best = scor
        depth_best = depth
        
print('Лучший скор:', scor_best, 'При depth:', depth_best)

0.47425334397304325 4
0.49975442629023614 5
Лучший скор: 0.49975442629023614 При depth: 5


**Проверка модели на валидационных данных:**

In [33]:
model = DecisionTreeClassifier(random_state=12345, max_depth=depth_best)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("f1:", f1_score(predicted_valid, target_valid))

f1: 0.5111241217798594


**Проверка на тестовых данных:**

In [34]:
predicted_test = model.predict(features_test)
print("f1:", f1_score(predicted_test, target_test))

f1: 0.5102086605339914


### RandomForestClassifier

**Подбор гиперпараметров:**

In [35]:
best_result = 0
best_est = 0
best_depth = 0
for est in range(1, 3):
    for depth in range (1, 3):


        pipe_lr = Pipeline([('tfidf', TfidfVectorizer(stop_words=stopwords)),
         ('model', RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth))])
        scor = (cross_val_score(pipe_lr, train['lemm'], train['toxic'], scoring='f1', cv=20)).mean()
        print(scor, est, depth)
        if scor > best_result:
            best_result = scor
            best_est = est
            best_depth = depth



print("Лучший скор:", best_result, "При количестве деревьев:", best_est, "Максимальной глубине:", best_depth)


0.014599688295364125 1 1
0.021175861152700003 1 2
0.010040321637912136 2 1
0.0164776450317159 2 2
Лучший скор: 0.021175861152700003 При количестве деревьев: 1 Максимальной глубине: 2


**Проверка модели на валидационных данных:**

In [36]:
model = RandomForestClassifier(random_state=12345, n_estimators=best_est, max_depth=best_depth)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("f1:", f1_score(predicted_valid, target_valid))

f1: 0.00878594249201278


**Проверка на тестовых данных:**

In [37]:
predicted_test = model.predict(features_test)
print("f1:", f1_score(predicted_test, target_test))

f1: 0.010705000764642912


## Вывод

Лучшее качество показала модель логистической регрессии с парметром С = 6.2.
Достигнуто качество в 0.76 на тестовых данных, чего для решения данной задачи - достаточно.

Вот такие дела.